#install

In [1]:
# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 
# Install Folium for Geographic data visualization
!pip install folium
# Install plotlyExpress
!pip install plotly_express
!pip install Shapely
!pip install plotly_express

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
The following additional packages will be installed:
  python3-numpy
Suggested packages:
  python-numpy-doc python3-nose python3-numpy-dbg
The following NEW packages will be installed:
  python3-gdal python3-numpy
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 2,288 kB of archives.
After this operation, 13.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python3-numpy amd64 1:1.13.3-2ubuntu1 [1,943 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python3-gdal amd64 2.2.3+dfsg-2 [346 kB]
Fetched 2,288 kB in 1s (1,918 kB/s)
Selecting previously unselected package python3-numpy.
(Reading database ... 155013 files and directories currently installed.)
Preparing to unpack .../python3-numpy_

In [2]:
from shapely.geometry import Point,Polygon
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt 
import folium
from folium import plugins
from folium.plugins import HeatMap
import plotly_express as px
from google.colab import drive
from IPython.display import display
import glob
import re
import plotly_express as px
import seaborn as sns
drive.mount('/content/drive')

Mounted at /content/drive


#Read

In [39]:
!ls '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/'

 District_NE.geojson	  gps_log_NE_2019-04	   roadDB_NE_3digit.csv
 gps_log_2019-04-week-1   Open_Street_Map	   route_network
 gps_log_2019-04-week-2   Province_boundary	   trips_and_stops_NE_2019-04
 gps_log_2019-04-week-3  'Road Data'
 gps_log_2019-04-week-4   roadDB_NE_1_2digit.csv


In [40]:
tripPath = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/trips_and_stops_NE_2019-04/trips_2019_4_8.csv'
districtPath = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/District_NE.geojson'

In [41]:
trip = pd.read_csv(tripPath)
district_NE = gpd.read_file(districtPath)

In [8]:
trip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 444395 entries, 0 to 444394
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   time_stamp  444395 non-null  object 
 1   unit_id     444395 non-null  object 
 2   lat         444395 non-null  float64
 3   lon         444395 non-null  float64
 4   speed       444395 non-null  int64  
 5   unit_type   444395 non-null  float64
 6   trip_id     444395 non-null  int64  
dtypes: float64(3), int64(2), object(2)
memory usage: 23.7+ MB


In [42]:
district_NE.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 321 entries, 0 to 320
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   AMP_CODE    321 non-null    object  
 1   PRV_CODE    321 non-null    object  
 2   AMP_NAME_T  321 non-null    object  
 3   AMP_NAME_E  321 non-null    object  
 4   Shape_Leng  321 non-null    float64 
 5   Shape_Area  321 non-null    float64 
 6   geometry    321 non-null    geometry
dtypes: float64(2), geometry(1), object(4)
memory usage: 17.7+ KB


In [46]:
trip[['time_stamp']] = trip[['time_stamp']].astype("datetime64")

In [11]:
trip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 444395 entries, 0 to 444394
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   time_stamp  444395 non-null  datetime64[ns]
 1   unit_id     444395 non-null  object        
 2   lat         444395 non-null  float64       
 3   lon         444395 non-null  float64       
 4   speed       444395 non-null  int64         
 5   unit_type   444395 non-null  float64       
 6   trip_id     444395 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(2), object(1)
memory usage: 23.7+ MB


In [24]:
trip

,time_stamp,unit_id,lat,lon,speed,unit_type,trip_id
0,2019-04-08 08:09:02,060000500000000011519719933,15.676680,101.993200,0,7.0,0
1,2019-04-08 08:09:07,060000500000000011519719933,15.676800,101.993330,0,7.0,0
2,2019-04-08 08:09:12,060000500000000011519719933,15.676800,101.993330,0,7.0,0
3,2019-04-08 08:09:17,060000500000000011519719933,15.676800,101.993330,0,7.0,0
4,2019-04-08 08:09:22,060000500000000011519719933,15.676800,101.993330,0,7.0,0
...,...,...,...,...,...,...,...
444390,2019-04-08 08:00:10,002000400000000000000521991,17.271763,102.323932,48,8.0,11709
444391,2019-04-08 08:22:00,002000400000000000000521991,17.310813,102.175172,44,8.0,11709
444392,2019-04-08 08:42:00,002000400000000000000521991,17.325908,102.062842,46,8.0,11709
444393,2019-04-08 09:14:51,002000400000000000000521991,17.365932,101.912222,34,8.0,11709


In [32]:
trip

,time_stamp,unit_id,lat,lon,speed,unit_type,trip_id
0,2019-04-08 08:09:02,060000500000000011519719933,15.676680,101.993200,0,7.0,0
1,2019-04-08 08:09:07,060000500000000011519719933,15.676800,101.993330,0,7.0,0
2,2019-04-08 08:09:12,060000500000000011519719933,15.676800,101.993330,0,7.0,0
3,2019-04-08 08:09:17,060000500000000011519719933,15.676800,101.993330,0,7.0,0
4,2019-04-08 08:09:22,060000500000000011519719933,15.676800,101.993330,0,7.0,0
...,...,...,...,...,...,...,...
444390,2019-04-08 08:00:10,002000400000000000000521991,17.271763,102.323932,48,8.0,11709
444391,2019-04-08 08:22:00,002000400000000000000521991,17.310813,102.175172,44,8.0,11709
444392,2019-04-08 08:42:00,002000400000000000000521991,17.325908,102.062842,46,8.0,11709
444393,2019-04-08 09:14:51,002000400000000000000521991,17.365932,101.912222,34,8.0,11709


#test

In [16]:
test01 = trip.loc[115]

In [23]:
test01

time_stamp            2019-04-08 02:46:04
unit_id       047000100000864507032291287
lat                               14.6372
lon                               101.257
speed                                  60
unit_type                               7
trip_id                                 1
Name: 115, dtype: object

In [22]:
test01.time_stamp.day_name()

'Monday'

#Algorithm

In [43]:
def check_district(lon,lat):
  point = Point(lon,lat)
  for index in district_NE.index:
    if point.within(district_NE.loc[index,'geometry']):
      return True
  return False

In [44]:
check_district(103.423698,15.916181)

True

#Featue

In [47]:
for index in trip.index:
  trip['day'] = trip.loc[index,'time_stamp'].day_name()

KeyboardInterrupt: ignored